In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pathlib import Path

# Load environment variables from .env file
current_dir = os.getcwd()
env_path = os.path.join(current_dir, '.env')
# Access your API key from the .env file
load_dotenv(dotenv_path=env_path)  
api_key = os.getenv("API_KEY")
# Initialize the model with the API key
model = ChatOpenAI(model="llama-3.3-70b-versatile", api_key=api_key, base_url="https://api.groq.com/openai/v1")
print(api_key)
print(env_path)

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
c:\Users\Admin-62501\Desktop\Coding\Projects\Customer agents for market simulation\agent\.env


我們也可以自己寫agent: https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/

langgraph swarm: https://langchain-ai.github.io/langgraph/reference/swarm/

`config` object in LangGraph Swarm, specifically the {"configurable": {"thread_id": "1"}} structure, is not modified by the system during execution. It serves as a persistent identifier that you provide to maintain conversation state across multiple interactions

`router` in `StateGraph`: router refers to a special type of node or function that determines the flow of execution through the graph based on the current state. It acts as a decision-making component that directs which node should be executed next.

In [2]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

In [ ]:


def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

alice = create_react_agent(
    model,
    [add, create_handoff_tool(agent_name="Bob")],
    prompt="You are Alice, an addition expert.",
    name="Alice",
)

bob = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Alice", description="Transfer to Alice, she can help with math")],
    prompt="You are Bob, you speak like a pirate.",
    name="Bob",
)
# short-term memory
# maintain conversation state across interactions
checkpointer = InMemorySaver()
# long-term memory
store = InMemoryStore()

workflow = create_swarm(
    [alice, bob],
    default_active_agent="Alice"
)

# Compiles the state graph into a CompiledStateGraph object.
app = workflow.compile(
    checkpointer=checkpointer,
    store=store
)
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "i'd like to speak to Bob"}]},
    config,
)
print(turn_1)
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
    config,
)
print(turn_2)

{'messages': [HumanMessage(content="i'd like to speak to Bob", additional_kwargs={}, response_metadata={}, id='b35e6d5a-3a62-46c2-8ca8-0a4b945b3b94'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2966', 'function': {'arguments': '{}', 'name': 'transfer_to_bob'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 275, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.262570968, 'prompt_time': 0.024186555, 'completion_time': 0.043636364, 'total_time': 0.067822919}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'id': 'chatcmpl-2324f17b-a548-40ad-a1e7-c760ebfb415d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, name='Alice', id='run--6bd1fe6e-6074-4935-80da-7399e5f7cf0a-0', tool_calls=[{'name': 'transfer_to_bob', 'args': {}, 'id': 'call_2966', 'type': 'tool_call'}], usage_metadata={'input_t

Try to add orchestrator agent

In [ ]:
orchestrator = create_react_agent(
    model,
    [create_handoff_tool(agent_name="alice"), create_handoff_tool(agent_name="bob")],
    prompt="""You are a meta-expert orchestrator. Your job is to:
    1. Analyze the user's request
    2. Plan which agents should handle different aspects of the task
    3. Create specific instructions for each agent using meta-prompting
    4. Hand off to the appropriate agent
    When all subtasks are complete, hand off to the synthesizer.""",
    name="orchestrator"
)

alice = create_react_agent(
    model,
    [add, create_handoff_tool(agent_name="orchestrator"), 
     create_handoff_tool(agent_name="synthesizer")],
    prompt="You are Alice, an addition expert. Follow the specific instructions provided by the orchestrator.",
    name="alice"
)

bob = create_react_agent(
    model,
    [create_handoff_tool(agent_name="orchestrator"), 
     create_handoff_tool(agent_name="synthesizer")],
    prompt="You are Bob, you speak like a pirate. Follow the specific instructions provided by the orchestrator.",
    name="bob"
)

synthesizer = create_react_agent(
    model,
    [create_handoff_tool(agent_name="orchestrator")],
    prompt="""You are a synthesizer agent. Your job is to:
    1. Review all the work done by other agents
    2. Combine their outputs into a coherent, comprehensive response
    3. Ensure the final answer addresses all aspects of the original query""",
    name="synthesizer"
)



agent dynamically generation 

In [10]:
# Persona Generator Agent
persona_generator = create_react_agent(
    model,
    [create_handoff_tool(agent_name="discussion_coordinator")],
    prompt="""You are a Persona Generator. Your task is to create 2-3 distinct customer personas who would be interested in purchasing a specific product.
        For each persona, include:
        - A brief, descriptive name (e.g., "Tech-Savvy Tony," "Budget-Conscious Brenda")
        - Key demographic information (age range, occupation, general lifestyle)
        - Their primary motivations and needs related to this type of product
        - Any potential concerns or hesitations they might have

        Format your response as a JSON array of persona objects. Each object should have:
        - name: The persona's descriptive name
        - demographics: Key demographic information
        - motivations: Primary motivations and needs
        - concerns: Potential concerns or hesitations

        Example:
        [
        {
        "name": "Tech-Savvy Tony",
        "demographics": "35-45, IT professional, urban lifestyle with high disposable income",
        "motivations": "Wants cutting-edge features, values innovation and performance",
        "concerns": "Worried about product becoming obsolete too quickly, needs seamless integration with existing devices"
        },
        {
        "name": "Budget-Conscious Brenda",
        "demographics": "28-40, mid-level manager, suburban family lifestyle",
        "motivations": "Seeks value for money, durability, and practical features",
        "concerns": "Price point, long-term maintenance costs, ease of use for family members"
        }
        ]
        When given a product name, generate appropriate personas who would realistically be interested in that product.
        """,
    name="persona_generator"
)

disscussion_coordinator = create_react_agent(
    model,
    [create_handoff_tool(agent_name="persona_generator")],
    prompt="...",
    name="discussion_coordinator"
)

# short-term memory
# maintain conversation state across interactions
checkpointer = InMemorySaver()
# long-term memory
store = InMemoryStore()

config = {"configurable": {"thread_id": "1"}}

workflow = create_swarm(
    [persona_generator, disscussion_coordinator],
    default_active_agent="persona_generator"
)

# Compiles the state graph into a CompiledStateGraph object.
app = workflow.compile(
    checkpointer=checkpointer,
    store=store
)

Interactive agents

In [11]:
def format_agent_response(response):
    """Format the agent response for better readability."""
    agent_name = response.get("active_agent", "Unknown")
    messages = response.get("messages", [])

    if not messages:
        return f"[{agent_name}] No response."

    last_message = messages[-1]
    content = getattr(last_message, 'content', "")  # Access 'content' attribute directly

    return f"[{agent_name}] {content}"

def interactive_chat():
    """Run an interactive chat session with the agent swarm"""
    print("Welcome to the Agent Swarm Chat!")
    print("Type 'exit' or 'quit' to end the conversation.")
    print("Starting with Alice as the default agent.")
    print("-" * 50)
    
    while True:
        # Get user input
        user_input = input("You: ")
        
        # Check if user wants to exit
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        
        # Create a message with just the current user input
        # The checkpointer will maintain the conversation state
        current_message = [{"role": "user", "content": user_input}]
        
        # Invoke the agent swarm with just the new message
        response = app.invoke(
            {"messages": current_message},
            config,
        )
        
        # Extract and display the agent's response
        formatted_response = format_agent_response(response)
        print(formatted_response)


In [12]:
interactive_chat()

Welcome to the Agent Swarm Chat!
Type 'exit' or 'quit' to end the conversation.
Starting with Alice as the default agent.
--------------------------------------------------
[Unknown] [
{
"name": "Sweet-Toothed Sally",
"demographics": "18-30, student or young professional, active social life",
"motivations": "Craves unique and exciting flavors, values convenience and affordability",
"concerns": "Calorie intake, potential allergens or sensitivities, limited dietary options"
},
{
"name": "Family-Focused Frank",
"demographics": "30-50, parent or guardian, suburban family lifestyle",
"motivations": "Wants to provide treats for family members, values variety packs and bulk options",
"concerns": "Added sugars, artificial ingredients, and potential choking hazards for young children"
},
{
"name": "Nostalgic Nancy",
"demographics": "40-60, established career, nostalgic for childhood treats",
"motivations": "Seeks classic candy flavors and textures, values sentimental value and retro packaging",